In [1]:
# 평가지표 rmse
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, KFold
from sklearn.experimental import enable_halving_search_cv
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, MaxAbsScaler, PowerTransformer, PolynomialFeatures, QuantileTransformer
import warnings
from sklearn.preprocessing import LabelEncoder

In [2]:

path = "C:\\Users\\Judy\\Desktop\\house_predict\\"
train = pd.read_csv(path + 'train.csv')

test = pd.read_csv(path + 'test.csv')
submit_file = pd.read_csv(path + 'sample_submission.csv')

In [3]:
print(train.info())
print(train.shape) # (1460, 81)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [4]:
print(test.info())
print(test.shape) # (1459, 80)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1459 non-null   int64  
 1   MSSubClass     1459 non-null   int64  
 2   MSZoning       1455 non-null   object 
 3   LotFrontage    1232 non-null   float64
 4   LotArea        1459 non-null   int64  
 5   Street         1459 non-null   object 
 6   Alley          107 non-null    object 
 7   LotShape       1459 non-null   object 
 8   LandContour    1459 non-null   object 
 9   Utilities      1457 non-null   object 
 10  LotConfig      1459 non-null   object 
 11  LandSlope      1459 non-null   object 
 12  Neighborhood   1459 non-null   object 
 13  Condition1     1459 non-null   object 
 14  Condition2     1459 non-null   object 
 15  BldgType       1459 non-null   object 
 16  HouseStyle     1459 non-null   object 
 17  OverallQual    1459 non-null   int64  
 18  OverallC

In [5]:
#train.isnull()
train.isnull().sum() 

Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
                ... 
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
SalePrice          0
Length: 81, dtype: int64

In [6]:
test.isnull().sum() 

Id                 0
MSSubClass         0
MSZoning           4
LotFrontage      227
LotArea            0
                ... 
MiscVal            0
MoSold             0
YrSold             0
SaleType           1
SaleCondition      0
Length: 80, dtype: int64

In [7]:
means = train.mean()
print(means)

Id                  730.500000
MSSubClass           56.897260
LotFrontage          70.049958
LotArea           10516.828082
OverallQual           6.099315
OverallCond           5.575342
YearBuilt          1971.267808
YearRemodAdd       1984.865753
MasVnrArea          103.685262
BsmtFinSF1          443.639726
BsmtFinSF2           46.549315
BsmtUnfSF           567.240411
TotalBsmtSF        1057.429452
1stFlrSF           1162.626712
2ndFlrSF            346.992466
LowQualFinSF          5.844521
GrLivArea          1515.463699
BsmtFullBath          0.425342
BsmtHalfBath          0.057534
FullBath              1.565068
HalfBath              0.382877
BedroomAbvGr          2.866438
KitchenAbvGr          1.046575
TotRmsAbvGrd          6.517808
Fireplaces            0.613014
GarageYrBlt        1978.506164
GarageCars            1.767123
GarageArea          472.980137
WoodDeckSF           94.244521
OpenPorchSF          46.660274
EnclosedPorch        21.954110
3SsnPorch             3.409589
ScreenPo

C:\Users\Judy\AppData\Local\Temp\ipykernel_16260\3159416517.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  means = train.mean()


In [8]:
train_data = train.fillna(means)
#print(train_data)
train_data.isnull().sum()

Id               0
MSSubClass       0
MSZoning         0
LotFrontage      0
LotArea          0
                ..
MoSold           0
YrSold           0
SaleType         0
SaleCondition    0
SalePrice        0
Length: 81, dtype: int64

In [9]:
meanss = test.mean()
print(meanss)

Id               2190.000000
MSSubClass         57.378341
LotFrontage        68.580357
LotArea          9819.161069
OverallQual         6.078821
OverallCond         5.553804
YearBuilt        1971.357779
YearRemodAdd     1983.662783
MasVnrArea        100.709141
BsmtFinSF1        439.203704
BsmtFinSF2         52.619342
BsmtUnfSF         554.294925
TotalBsmtSF      1046.117970
1stFlrSF         1156.534613
2ndFlrSF          325.967786
LowQualFinSF        3.543523
GrLivArea        1486.045922
BsmtFullBath        0.434454
BsmtHalfBath        0.065202
FullBath            1.570939
HalfBath            0.377656
BedroomAbvGr        2.854010
KitchenAbvGr        1.042495
TotRmsAbvGrd        6.385195
Fireplaces          0.581220
GarageYrBlt      1977.721217
GarageCars          1.766118
GarageArea        472.768861
WoodDeckSF         93.174777
OpenPorchSF        48.313914
EnclosedPorch      24.243317
3SsnPorch           1.794380
ScreenPorch        17.064428
PoolArea            1.744345
MiscVal       

C:\Users\Judy\AppData\Local\Temp\ipykernel_16260\4050549393.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  meanss = test.mean()


In [10]:
test_data = test.fillna(meanss)
#print(train_data)
test_data.isnull().sum()

Id               0
MSSubClass       0
MSZoning         4
LotFrontage      0
LotArea          0
                ..
MiscVal          0
MoSold           0
YrSold           0
SaleType         1
SaleCondition    0
Length: 80, dtype: int64

In [11]:
# # 이상치 제거
# def remove_outlier(input_data):
#     q1 = input_data.quantile(0.25) # 제 1사분위수
#     q3 = input_data.quantile(0.75) # 제 3사분위수
#     iqr = q3 - q1 # IQR(Interquartile range) 계산
#     minimum = q1 - (iqr * 1.5) # IQR 최솟값
#     maximum = q3 + (iqr * 1.5) # IQR 최댓값
#     ### IQR 범위 내에 있는 데이터만 산출(IQR 범위 밖의 데이터는 이상치) ###
#     df_removed_outlier = input_data[(minimum < input_data) & (input_data < maximum)]
#     return df_removed_outlier

In [12]:
# train_data = remove_outlier(train_data)
# test_data = remove_outlier(test_data)

In [13]:
# train_data = train_data.interpolate()
# test_data = test_data.interpolate()

In [14]:
#print(train_data.isnull())
print(train_data.isnull().sum())
print(test_data.isnull().sum())


Id               0
MSSubClass       0
MSZoning         0
LotFrontage      0
LotArea          0
                ..
MoSold           0
YrSold           0
SaleType         0
SaleCondition    0
SalePrice        0
Length: 81, dtype: int64
Id               0
MSSubClass       0
MSZoning         4
LotFrontage      0
LotArea          0
                ..
MiscVal          0
MoSold           0
YrSold           0
SaleType         1
SaleCondition    0
Length: 80, dtype: int64


In [15]:
print(train_data.corr())


                     Id  MSSubClass  LotFrontage   LotArea  OverallQual  \
Id             1.000000    0.011156    -0.009601 -0.033226    -0.028365   
MSSubClass     0.011156    1.000000    -0.357056 -0.139781     0.032628   
LotFrontage   -0.009601   -0.357056     1.000000  0.306795     0.234196   
LotArea       -0.033226   -0.139781     0.306795  1.000000     0.105806   
OverallQual   -0.028365    0.032628     0.234196  0.105806     1.000000   
OverallCond    0.012609   -0.059316    -0.052820 -0.005636    -0.091932   
YearBuilt     -0.012713    0.027850     0.117598  0.014228     0.572323   
YearRemodAdd  -0.021998    0.040581     0.082746  0.013788     0.550684   
MasVnrArea    -0.050199    0.022895     0.179283  0.103960     0.410238   
BsmtFinSF1    -0.005024   -0.069836     0.215828  0.214103     0.239666   
BsmtFinSF2    -0.005968   -0.065649     0.043340  0.111170    -0.059119   
BsmtUnfSF     -0.007940   -0.140759     0.122156 -0.002618     0.308159   
TotalBsmtSF   -0.015415  

In [16]:
# sns.set(font_scale=1.0)
# sns.heatmap(data=train_data.corr(), square=True, annot=True, cbar=True)
# plt.show()

In [17]:
le = LabelEncoder()
train_data['MSZoning'] = le.fit_transform(train_data['MSZoning'])
train_data['Street'] = le.fit_transform(train_data['Street'])
train_data['Alley'] = le.fit_transform(train_data['Alley'])
train_data['LotShape'] = le.fit_transform(train_data['LotShape'])
train_data['LandContour'] = le.fit_transform(train_data['LandContour'])
train_data['Utilities'] = le.fit_transform(train_data['Utilities'])
train_data['LotConfig'] = le.fit_transform(train_data['LotConfig'])
train_data['LandSlope'] = le.fit_transform(train_data['LandSlope'])
train_data['Neighborhood'] = le.fit_transform(train_data['Neighborhood'])
train_data['Condition1'] = le.fit_transform(train_data['Condition1'])
train_data['Condition2'] = le.fit_transform(train_data['Condition2'])
train_data['BldgType'] = le.fit_transform(train_data['BldgType'])
train_data['HouseStyle'] = le.fit_transform(train_data['HouseStyle'])
train_data['SaleCondition'] = le.fit_transform(train_data['SaleCondition'])
train_data['RoofStyle'] = le.fit_transform(train_data['RoofStyle'])
train_data['RoofMatl'] = le.fit_transform(train_data['RoofMatl'])
train_data['Exterior1st'] = le.fit_transform(train_data['Exterior1st'])
train_data['Exterior2nd'] = le.fit_transform(train_data['Exterior2nd'])
train_data['MasVnrType'] = le.fit_transform(train_data['MasVnrType'])
train_data['ExterQual'] = le.fit_transform(train_data['ExterQual'])
train_data['ExterCond'] = le.fit_transform(train_data['ExterCond'])
train_data['Foundation'] = le.fit_transform(train_data['Foundation'])
train_data['BsmtQual'] = le.fit_transform(train_data['BsmtQual'])
train_data['BsmtCond'] = le.fit_transform(train_data['BsmtCond'])
train_data['BsmtExposure'] = le.fit_transform(train_data['BsmtExposure'])
train_data['BsmtFinType1'] = le.fit_transform(train_data['BsmtFinType1'])
train_data['BsmtFinType2'] = le.fit_transform(train_data['BsmtFinType2'])
train_data['Heating'] = le.fit_transform(train_data['Heating'])
train_data['HeatingQC'] = le.fit_transform(train_data['HeatingQC'])
train_data['CentralAir'] = le.fit_transform(train_data['CentralAir'])
train_data['Electrical'] = le.fit_transform(train_data['Electrical'])
train_data['KitchenQual'] = le.fit_transform(train_data['KitchenQual'])
train_data['Functional'] = le.fit_transform(train_data['Functional'])
train_data['FireplaceQu'] = le.fit_transform(train_data['FireplaceQu'])
train_data['GarageType'] = le.fit_transform(train_data['GarageType'])
train_data['GarageFinish'] = le.fit_transform(train_data['GarageFinish'])
train_data['GarageQual'] = le.fit_transform(train_data['GarageQual'])
train_data['GarageCond'] = le.fit_transform(train_data['GarageCond'])
train_data['PavedDrive'] = le.fit_transform(train_data['PavedDrive'])
train_data['PoolQC'] = le.fit_transform(train_data['PoolQC'])
train_data['Fence'] = le.fit_transform(train_data['Fence'])
train_data['MiscFeature'] = le.fit_transform(train_data['MiscFeature'])
train_data['MiscVal'] = le.fit_transform(train_data['MiscVal'])
train_data['SaleType'] = le.fit_transform(train_data['SaleType'])
train_data['SaleCondition'] = le.fit_transform(train_data['SaleCondition'])

test_data['MSZoning'] = le.fit_transform(test_data['MSZoning'])
test_data['Street'] = le.fit_transform(test_data['Street'])
test_data['Alley'] = le.fit_transform(test_data['Alley'])
test_data['LotShape'] = le.fit_transform(test_data['LotShape'])
test_data['LandContour'] = le.fit_transform(test_data['LandContour'])
test_data['Utilities'] = le.fit_transform(test_data['Utilities'])
test_data['LotConfig'] = le.fit_transform(test_data['LotConfig'])
test_data['LandSlope'] = le.fit_transform(test_data['LandSlope'])
test_data['Neighborhood'] = le.fit_transform(test_data['Neighborhood'])
test_data['Condition1'] = le.fit_transform(test_data['Condition1'])
test_data['Condition2'] = le.fit_transform(test_data['Condition2'])
test_data['BldgType'] = le.fit_transform(test_data['BldgType'])
test_data['HouseStyle'] = le.fit_transform(test_data['HouseStyle'])
test_data['SaleCondition'] = le.fit_transform(test_data['SaleCondition'])
test_data['RoofStyle'] = le.fit_transform(test_data['RoofStyle'])
test_data['RoofMatl'] = le.fit_transform(test_data['RoofMatl'])
test_data['Exterior1st'] = le.fit_transform(test_data['Exterior1st'])
test_data['Exterior2nd'] = le.fit_transform(test_data['Exterior2nd'])
test_data['MasVnrType'] = le.fit_transform(test_data['MasVnrType'])
test_data['ExterQual'] = le.fit_transform(test_data['ExterQual'])
test_data['ExterCond'] = le.fit_transform(test_data['ExterCond'])
test_data['Foundation'] = le.fit_transform(test_data['Foundation'])
test_data['BsmtQual'] = le.fit_transform(test_data['BsmtQual'])
test_data['BsmtCond'] = le.fit_transform(test_data['BsmtCond'])
test_data['BsmtExposure'] = le.fit_transform(test_data['BsmtExposure'])
test_data['BsmtFinType1'] = le.fit_transform(test_data['BsmtFinType1'])
test_data['BsmtFinType2'] = le.fit_transform(test_data['BsmtFinType2'])
test_data['Heating'] = le.fit_transform(test_data['Heating'])
test_data['HeatingQC'] = le.fit_transform(test_data['HeatingQC'])
test_data['CentralAir'] = le.fit_transform(test_data['CentralAir'])
test_data['Electrical'] = le.fit_transform(test_data['Electrical'])
test_data['KitchenQual'] = le.fit_transform(test_data['KitchenQual'])
test_data['Functional'] = le.fit_transform(test_data['Functional'])
test_data['FireplaceQu'] = le.fit_transform(test_data['FireplaceQu'])
test_data['GarageType'] = le.fit_transform(test_data['GarageType'])
test_data['GarageFinish'] = le.fit_transform(test_data['GarageFinish'])
test_data['GarageQual'] = le.fit_transform(test_data['GarageQual'])
test_data['GarageCond'] = le.fit_transform(test_data['GarageCond'])
test_data['PavedDrive'] = le.fit_transform(test_data['PavedDrive'])
test_data['PoolQC'] = le.fit_transform(test_data['PoolQC'])
test_data['Fence'] = le.fit_transform(test_data['Fence'])
test_data['MiscFeature'] = le.fit_transform(test_data['MiscFeature'])
test_data['MiscVal'] = le.fit_transform(test_data['MiscVal'])
test_data['SaleType'] = le.fit_transform(test_data['SaleType'])
test_data['SaleCondition'] = le.fit_transform(test_data['SaleCondition'])


train_data.dtypes
#train_data = train_data.astype('float')
#train_data.dtypes

Id                 int64
MSSubClass         int64
MSZoning           int32
LotFrontage      float64
LotArea            int64
                  ...   
MoSold             int64
YrSold             int64
SaleType           int32
SaleCondition      int64
SalePrice          int64
Length: 81, dtype: object

In [18]:
#train_data.info()

In [19]:
x = train_data.drop(['Id', 'SalePrice', 'MSZoning','KitchenAbvGr'], axis = 1)
y = train_data['SalePrice']
test = test_data.drop(['Id','MSZoning','KitchenAbvGr'], axis =1)

In [20]:
print(x.shape, y.shape) # (1460, 77) (1460,)

#print(np.unique(y)) #회귀

(1460, 77) (1460,)


In [21]:
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=66, shuffle=True, train_size = 0.8)

In [22]:
scaler = PolynomialFeatures()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
test = scaler.transform(test)

In [23]:
n_splits = 5
kfold = KFold(n_splits = n_splits, shuffle = True, random_state = 66)

#parameters = [
 #   {'n_estimators' : [100, 200, 300], 'max_depth' : [6, 8, 10, 12]},
 #   {'max_depth' : [6, 8, 10, 12],'min_samples_leaf' : [3, 5, 7,10], 'min_samples_leaf' : [3,5,7,10]},
 #   {'min_samples_leaf' : [3, 5, 7, 10], 'min_samples_split' : [2, 3, 5, 10]},
 #  {'min_samples_split' : [2, 3, 5, 10], 'max_depth' : [6,8,10,12]}]   

parameters = [
    {'n_estimators' : [100, 200, 300], 'max_depth' : [6, 8, 10]},
    {'max_depth' : [6, 8, 10, 12],'min_samples_leaf' : [3, 5, 7], 'min_samples_leaf' : [3,5,7,10]},
    {'min_samples_leaf' : [3, 5, 7, 10], 'min_samples_split' : [2, 3, 5, 10]},
    {'min_samples_split' : [2, 3, 5, 10], 'max_depth' : [6,8,10]}]   

In [24]:
model = RandomizedSearchCV(RandomForestRegressor(), parameters, cv = kfold, verbose=1, refit=True, n_jobs=-1, n_iter = 20) 
#model = GridSearchCV(RandomForestRegressor(), parameters, cv = kfold, verbose=1, refit=True, n_jobs=-1) 

In [25]:
import time
start = time.time()
model.fit(x_train, y_train)
end = time.time() - start

Fitting 5 folds for each of 20 candidates, totalling 100 fits


In [26]:
print(model.feature_importances_)

AttributeError: 'RandomizedSearchCV' object has no attribute 'feature_importances_'

In [ ]:
y_predict = model.predict(x_test)
print(mean_squared_error(y_test, y_predict, squared = False)) # round함수..

26142.422864584827


In [ ]:
# 제출
result = model.predict(test)
#print(result)
submit_file['SalePrice']= result
submit_file.to_csv(path + 'sample_submission.csv',index=False)